# azimuth venues

In [375]:
import os
import supabase
import pandas as pd
from postgrest import APIError
from supabase import create_client, Client

supabase_url: str = os.getenv("SUPABASE_URL")
supabase_key: str = os.getenv("SUPABASE_KEY")
supabase_client: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

---------------------------------------------------------------
    venues from seatgeek database

In [347]:
response = supabase_client.table('seatgeek_data').select("*").execute()
def seatGeek(response):
    # Create a DataFrame from the API response data
    df = pd.DataFrame(response.data)

    # Reset the index
    df = df.reset_index(drop=True)

    # Remove duplicates based on 'venue.name' column
    venue = df[~df['venue.name'].duplicated()]

    # Get the columns of interest
    cols = list(venue.columns)
    data = venue[cols[27:-12]]

    return data

In [348]:
# seatGeek(response)

----------------------------------------------------------
    _venues from ticketmaster database

In [349]:
response2 = supabase_client.table('ticketmasterdata').select("*").execute()
def ticketMaster(response):
    # Create a DataFrame from the API response data
    tkMaster_df = pd.DataFrame(response.data)

    # Reset the index
    tkMaster_df = tkMaster_df.reset_index(drop=True)

    # Select the columns of interest
    tkMaster_df = tkMaster_df.iloc[:, 17:-7]

    # Remove duplicates based on 'Venue Name' column
    tkMaster_df = tkMaster_df[~tkMaster_df['Venue Name'].duplicated()]

    return tkMaster_df


In [350]:
# ticketMaster(response2)

---------------------------------------------------------------------
    Venues.bson file from crankDB

In [351]:
def processVenuesData(filename):
    # Read the CSV file into a DataFrame
    venues = pd.read_csv(filename)

    # Remove duplicates based on 'Name' column
    venues = venues[~venues['Name'].duplicated()]

    # Drop unnecessary columns
    venues.drop(['MetroArea._id', '_id'], axis=1, inplace=True)

    return venues


In [352]:
# processVenuesData('venues.csv')

----------------------------------------------------
    chceking if there exist similarity between the unique values in tkmaster and crankdb
    
--------------------------------------------

    checking if the values in ticketmaster db exists in seatgeek

In [361]:
common_vanues = processVenuesData('venues.csv')['Name'].isin(seatGeek(response)['venue.name'])
crankdb_nique = processVenuesData('venues.csv')[~common_vanues]
print(crankdb_nique.shape)

comon = ticketMaster(response2)['Venue Name'].isin(crankdb_unique['name'])
tkMaster_unique = ticketMaster(response2)[comon]
print(tkMaster_unique.shape)

/tmp/ipykernel_10454/630730371.py:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  venues = pd.read_csv(filename)
/tmp/ipykernel_10454/630730371.py:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  venues = pd.read_csv(filename)


(126749, 20)
(33, 6)


In [362]:
crankdb_unique.shape, seatGeek(response).shape, tkMaster_unique.shape

((126749, 14), (3054, 30), (33, 6))

In [363]:
seatGeek = seatGeek(response)

In [376]:
import pandas as pd

def processSeatGeekData(seatGeek):
    # Rename columns
    cols = list(seatGeek.columns)
    seatGeek.columns = [i.replace('venue.', '').lower() for i in cols]

    # Select desired columns
    seatGeek = seatGeek[['name', 'state', 'country', 'postal_code', 'city', 'capacity', 'popularity', 'address', 'url', 'timezone', 'location.lat', 'location.lon']]

    # Rename columns
    seatGeek.columns = ['name', 'state', 'country', 'postalCode', 'city', 'capacity', 'popularity', 'address', 'url', 'timezone', 'longitude', 'latitude']

    # Add new columns
    seatGeek['description'] = None
    seatGeek['phone'] = ''
    seatGeek['metroareaName'] = None

    return seatGeek

seatGeek = processSeatGeekData(seatGeek)

In [369]:
crankdb_unique['postal_code'] = None
crankdb_unique['timezone'] = None
crankdb_unique.columns

Index(['name', 'state', 'country', 'city', 'capacity', 'address',
       'metroarea_name', 'phone', 'url', 'description', 'longitude',
       'latitude', 'postal_code', 'timezone'],
      dtype='object')

---------------------------

    joining the datasets
 ---------------------

In [372]:
azimuthVenues = pd.concat((seatGeek, crankdb_unique), axis = 0)

In [374]:
azimuthVenues.shape

(129803, 17)

----------------------------------------------------------

    inserting the following data into our database table
----------------------------------------------

In [ ]:
def loadToDb(azimuthVenues, supabase_client):
    # Replace NaN values with None
    azimuthVenues.replace({pd.np.nan: None}, inplace=True)

    # Convert DataFrame to JSON records
    json_data = azimuthVenues.to_dict(orient='records')

    # Insert records into the database
    for record in json_data:
        try:
            supabase_client.table('azimuth_venues').insert(record).execute()
        except APIError as e:
            print("Error occurred while inserting the record:")
            print(record)
            print("Error message:", e.message)
            print()

loadToDb(azimuthVenues, supabase_client)

--------------------------------------

    removing duplicated rows
-------------------------------

In [ ]:
def filterDups(supabase_client):
    response = supabase_client.table('azimuth_venues').select('*').execute()
    data = response.data

    # Create a set to store unique IDs
    unique_ids = set()

    # List to store rows to be deleted
    rows_to_delete = []

    # Iterate through each row and filter duplicates based on 'name'
    for row in data:
        row_name = row['name']
        if row_name not in unique_ids:
            unique_ids.add(row_name)
        else:
            rows_to_delete.append(row_name)

    # Delete the duplicate rows from the table
    for row_name in rows_to_delete:
        # Retrieve the first occurrence of the duplicate row
        first_occurrence = next(row for row in data if row['name'] == row_name)
        # Delete all other occurrences of the duplicate row
        supabase_client.table('azimuth_venues').delete().match({'name': row_name}).execute()
        # Insert the first occurrence back into the table
        supabase_client.table('azimuth_venues').insert(first_occurrence).execute()

    print("Duplicate rows retained and others deleted successfully.")

filterDups(supabase_client)